# Data Collection and Retrieval
Data Collection and Retrieval (same requirements as the individual data retrieval project):
* Topic: anything you choose, but it must include the following data types:
    * numeric features
    * categorical features (to be dummy coded)
    * text features (to be processed using text analytics)
    * image features (to be processed using image classification) 
    * labels to choose from (i.e. outcomes that you want to predict with the other features)
* Scrape approximately ~500 records; +/- 50

In [2]:
# !pip install pyLDAvis
# !pip install pyLDAvis.gensim
# !pip install logging
# !pip install nltk
# !pip install - U pip setuptools wheel
# !pip install - U spacy
# !python - m spacy download en_core_web_sm
# !pip install gower


In [5]:
import requests
import time
import json
import pandas as pd

def get_tweets(bearer_token):
    headers = {'Authorization': ('Bearer ' + bearer_token)}

    n = 525                               # The total number of tweets we want
    max_results = 10 # The number of tweets to pull per request; must be between 10 and 100
    total_retrieved = 0                   # To keep track of when to stop
    next_token = ""                       # Must be empty on first iteration
    search_term = "manchester%20united"

    # Create empty DataFrames and set columns
    df_tweets = pd.DataFrame(columns=['tweet_id', 'author_id', 'retweet_count', 'like_count',
                                      'text', 'language', 'created_at', 'source', 'possibly_sensitive', 'image_url'])

    # stop when we have n results
    while total_retrieved < n:

        # the first time through the loop, we do not need the next_token parameter
        if next_token == "":
            url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
        else:
            url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

        # These are the extra parameters we will add to the querystring; we won't store them all though; just want you to see what's possible
        url += f'&expansions=geo.place_id,author_id,attachments.media_keys'
        url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
        url += f'&media.fields=media_key,type,url&user.fields=created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

        # make the request to the Twitter API Recent Search endpoint
        response = requests.request("GET", url, headers=headers)
        try:  # Just in case we get an error
            json_data = json.loads(response.text)
            # print(json_data)
        except:
            print(response.text)

        for tweet in json_data['data']:
            media_key = ""  # Reset to empty each time through the loop so that we can use it for a condition later

            # Store the data into variables
            tweet_id = tweet['id']
            author_id = tweet['author_id']
            retweet_count = tweet['public_metrics']['retweet_count']    # label
            like_count = tweet['public_metrics']['like_count']          # label
            image_url = ""                                              # image
            text = tweet['text']                                        # text
            created_at = tweet['created_at']                            # categorical
            source = tweet['source']                                    # categorical
            possibly_sensitive = tweet['possibly_sensitive']            # categorical
            language = tweet['lang']                                    # categorical

            # Find out if there is media
            if 'attachments' in tweet:
                if 'media_keys' in tweet['attachments']:
                    media_key = tweet['attachments']['media_keys'][0]

            # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
            if media_key != "":
                for media in json_data['includes']['media']:
                    # Only if the media_key matches the one we stored
                    if media['media_key'] == media_key:
                        if media['type'] == 'photo':      # Only if it is a photo; ignore videos
                            # Store the url in a variable
                            image_url = media['url']


            # Add the new data to a new record in the DataFrame
            df_tweets.loc[tweet_id] = [tweet_id, author_id, retweet_count, like_count,
                                       text, language, created_at, source, possibly_sensitive, image_url]

        # keep track of how many results have been obtained so far:
        total_retrieved += 10
        print(f'{total_retrieved} tweets retrieved')

        # keep track of where to start next time, but quit if there are no more results
        try:
            next_token = json_data['meta']['next_token']
        except:
            break

        # sleep to avoid hitting the rate limit
        time.sleep(8)

    # All done! save the dataframes to csv files
    df_tweets.to_csv('tweets.csv')
    print('Got all the tweets!')
    return df_tweets

In [6]:
df = get_tweets(bearer_token='AAAAAAAAAAAAAAAAAAAAAGnaTwEAAAAAhRdM6yLmei6skyaWcjbx8IDFnlw%3DLPQHO2CTw1nVjjHLx3htgP9qmeCOgPpt96EdDujokNcWljI5iP')

10 tweets retrieved
20 tweets retrieved
30 tweets retrieved
40 tweets retrieved
50 tweets retrieved
60 tweets retrieved
70 tweets retrieved
80 tweets retrieved
90 tweets retrieved
100 tweets retrieved
110 tweets retrieved
120 tweets retrieved
130 tweets retrieved
140 tweets retrieved
150 tweets retrieved
160 tweets retrieved
170 tweets retrieved
180 tweets retrieved
190 tweets retrieved
200 tweets retrieved
210 tweets retrieved
220 tweets retrieved
230 tweets retrieved
240 tweets retrieved
250 tweets retrieved
260 tweets retrieved
270 tweets retrieved
280 tweets retrieved
290 tweets retrieved
300 tweets retrieved
310 tweets retrieved
320 tweets retrieved
330 tweets retrieved
340 tweets retrieved
350 tweets retrieved
360 tweets retrieved
370 tweets retrieved
380 tweets retrieved
390 tweets retrieved
400 tweets retrieved
410 tweets retrieved
420 tweets retrieved
430 tweets retrieved
440 tweets retrieved
450 tweets retrieved
460 tweets retrieved
470 tweets retrieved
480 tweets retrieved
4

# Feature Cleaning and Engineering
* The cleaning steps should include any basic steps needed to prepare the data including:
    * Binning rare group values
    * Standardizing values
    * Adjusting for skewness
    * Handle missing values
* These steps will be unique to each dataset
* Engineering includes converting the unstructured text and images features into usable features. This could include:
    * Topic moding text
    * Extracting characteristics of the text (e.g. word counts, sentiment)
    * Extracting characteristics of the images (e..g number of faces, smiles)

In [13]:
# Bin rare group values
def bin_groups(df, percent=.05, cols_to_exclude=[]):
    import pandas as pd
    for col in df:
        if col not in cols_to_exclude:
            if not pd.api.types.is_numeric_dtype(df[col]):
                for group, count in df[col].value_counts().iteritems():
                    if count / len(df) < percent:
                        df.loc[df[col] == group, col] = 'Other'
    return df

# Standardize data


def standardize_values(df):
    import pandas as pd
    import numpy as np
    from sklearn import preprocessing

    # Scale/normalize the features
    df = pd.DataFrame(preprocessing.MinMaxScaler(
    ).fit_transform(df), columns=df.columns)

    return df

# Dummy code categorical variables


def dummy_code_categorical_variables(df):
    import pandas as pd

    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = df.join(pd.get_dummies(
                df[col], prefix=col, drop_first=True, lsuffix='_left', rsuffix='_right'))

    return df

# Handle missing values


def drop_columns_missing_data(df, cutoff=.5):
    import pandas as pd
    for col in df:
        if df[col].isna().sum() / len(df) > cutoff:
            df.drop(columns=[col], inplace=True)
    return df

# Handle missing values


def impute_mean(df):
    from sklearn.impute import SimpleImputer
    import pandas as pd
    import numpy as np
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    imp = SimpleImputer(missing_values=np.nan, strategy='mean')
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df

# Handle missing values AND standardize values


def impute_KNN(df):
    from sklearn.impute import KNNImputer
    from sklearn.preprocessing import MinMaxScaler
    import pandas as pd
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    df = pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)
    imp = KNNImputer(n_neighbors=5, weights="uniform")
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df

# Handle missing values


def impute_reg(df):
    from sklearn.experimental import enable_iterative_imputer
    from sklearn.impute import IterativeImputer
    import pandas as pd
    for col in df:
        if not pd.api.types.is_numeric_dtype(df[col]):
            df = pd.get_dummies(df, columns=[col], drop_first=True)
    imp = IterativeImputer(max_iter=10, random_state=12345)
    df = pd.DataFrame(imp.fit_transform(df), columns=df.columns)
    return df


def fit_mlr(df, test_size=.2, random_state=12345, label=''):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import train_test_split
    import pandas as pd
    X = df.drop(label, axis=1)
    y = df[label]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state)
    model = LinearRegression().fit(X_train, y_train)
    print(f'R-squared (mlr): \t{model.score(X_test, y_test)}')
    return model


def fit_crossvalidate_mlr(df, k, label, repeat=True):
    from sklearn.linear_model import LinearRegression
    from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score
    import pandas as pd
    from numpy import mean, std
    X = df.drop(label, axis=1)
    y = df[label]
    if repeat:
        cv = RepeatedKFold(n_splits=10, n_repeats=5, random_state=12345)
    else:
        cv = KFold(n_splits=10, random_state=12345, shuffle=True)
    scores = cross_val_score(LinearRegression(), X, y,
                             scoring='r2', cv=cv, n_jobs=-1)
    print(f'Average R-squared:\t{mean(scores)}')
    return LinearRegression().fit(X, y)


def calc_sentiment(df):
    import pandas as pd
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer

    nltk.download('vader_lexicon')
    sia = SentimentIntensityAnalyzer()
    df['sentiment_overall'] = 0.0
    df['sentiment_negative'] = 0.0
    df['sentiment_neutral'] = 0.0
    df['sentiment_positive'] = 0.0

    for row in df.itertuples():
        sentiment = sia.polarity_scores(row.text)
        df.at[row.Index, 'sentiment_overall'] = sentiment['compound']
        df.at[row.Index, 'sentiment_negative'] = sentiment['neg']
        df.at[row.Index, 'sentiment_neutral'] = sentiment['neu']
        df.at[row.Index, 'sentiment_positive'] = sentiment['pos']

    # df.drop(columns=['Sentiment'], inplace=True) # I don't think this is necessary
    return df


def image_classification(df, api_key, api_secret):
    import pandas as pd
    import requests
    import json
    df_imagga = pd.DataFrame(columns=["interior objects", "nature landscape", "beaches seaside", "events parties", "food drinks",
                                      "paintings art", "pets animals", "text visuals", "sunrises sunsets", "cars vehicles",
                                      "macro flowers", "streetview architecture", "people portraits"])
    for row in df.itertuples():
        tweet_id = row.tweet_id
        if pd.isnull(row.image_url) or pd.isna(row.image_url):
            scores = [0.0] * len(df_imagga.columns)

            for n, col in enumerate(df_imagga.columns):
                # Iterate through each category of the result
                scores[n] = 0.0
                # Store the list as a new row in the DataFrame
                df_imagga.loc[tweet_id] = scores
        else:
            url = 'https://api.imagga.com/v2/categories/personal_photos/?image_url=' + row.image_url
            request = requests.get(url, auth=(api_key, api_secret))
            json_data = json.loads(request.text)

            # Create a list of 0.0 scores to update as we get data for each category we want to score in our DataFrame
            scores = [0.0] * len(df_imagga.columns)

            # Find the associated column in the DataFrame
            for n, col in enumerate(df_imagga.columns):
                # Iterate through each category of the result
                try:
                    for category in json_data["result"]["categories"]:
                        if col == category['name']['en']:
                            # Store the score
                            scores[n] = category['confidence']
                            break  # No need to keep looping once we've found the score

                except:
                    pass
                # Store the list as a new row in the DataFrame
                df_imagga.loc[tweet_id] = scores

    # merge the two DataFrames
    df = pd.merge(df, df_imagga, left_on=df.tweet_id, right_on=df_imagga.index)

    return df


def drop_random_columns(df):
    if 'Unnamed: 0' in df.columns:
        df.drop(columns=['Unnamed: 0'], inplace=True)
    if 'key_0' in df.columns:
        df.drop(columns=['key_0'], inplace=True)
    return df


def correct_skew(df):
    import numpy as np
    for col in df:
        if pd.api.types.is_numeric_dtype(df[col]):
            print(col + " Normal: " + str(df[col].skew()))
            if (df[col].skew() > 1) or (df[col].skew() < -1):
                log = np.log(df[col] + 1).skew()
                print(log)
                cbrt = (df[col]**(1/3)).skew()
                print(cbrt)
                sqrt = (df[col]**(1/2)).skew()
                print(sqrt)
                lowest = min(abs(log), abs(cbrt), abs(sqrt))
                print("Lowest: " + str(lowest))
                if abs(log) == lowest:
                    df[col] = np.log(df[col] + 1)
                elif abs(cbrt) == lowest:
                    df[col] = df[col]**(1/3)
                elif abs(sqrt) == lowest:
                    df[col] == df[col]**(1/2)
                print(col + " Fixed :" + str(df[col].skew()))
    for col in df:
        if pd.api.types.is_numeric_dtype(col):
            print(col + "Final: " + str(df[col].skew()))
    return df


def vif(df):
    import pandas as pd
    from sklearn.linear_model import LinearRegression

    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # drop the following columns with high VIF scores (check manually each time :))
    for col in df.drop(columns=['key_0', 'tweet_id', 'author_id', 'retweet_count', 'sentiment_neutral', 'sentiment_positive', 'sentiment_negative',
                                'cars vehicles', 'sunrises sunsets', 'macro flowers', 'text visuals', 'streetview architecture', 'pets animals',
                                'interior objects', 'nature landscape', 'beaches seaside', 'events parties', 'food drinks', 'paintings art', 'people portraits']):

        y = df[col]
        X = df.drop(columns=[col])

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        if r_squared < 1:  # Prevent division by zero runtime error
            vif = 1/(1 - r_squared)
        else:
            vif = 100

    df = df.drop(columns=['key_0', 'tweet_id', 'author_id', 'retweet_count', 'sentiment_neutral', 'sentiment_positive', 'sentiment_negative',
                              'cars vehicles', 'sunrises sunsets', 'macro flowers', 'text visuals', 'streetview architecture', 'pets animals',
                              'interior objects', 'nature landscape', 'beaches seaside', 'events parties', 'food drinks', 'paintings art', 'people portraits'])

    return df

In [15]:
import pandas as pd

df = pd.read_csv("./tweets.csv")

# Data cleaning and Prep
df = bin_groups(df, percent=.05, cols_to_exclude=['text', 'created_at', 'image_url']) # Exclude columns it doesn't make sense to bin

# Engineering
df = calc_sentiment(df)
df = image_classification(df=df, api_key='acc_81f8f349aa51d69', api_secret='dd79216824834fc2b208323818d1ca35')

# drop columns I'm finished analyzing
df = df.drop(columns=['text', 'created_at', 'image_url'])

# INSERT SKEW FUNCTION HERE
df = correct_skew(df)

# impute KNN, also standardize values
df = impute_KNN(df)

# INSERT VIF FUNCTION HERE
df = vif(df)

df


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\isaac\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


key_0 Normal: 0.0744580502425429
Unnamed: 0 Normal: 0.0744580502425429
tweet_id Normal: 0.0744580502425429
author_id Normal: -0.18743865947325786
retweet_count Normal: 1.9092850521431188
0.1397365521934738
0.8324213262340547
1.2949643560966675
Lowest: 0.1397365521934738
retweet_count Fixed :0.1397365521934738
like_count Normal: 9.443158157028593
4.9817986112504045
4.52378617696872
6.053212173058231
Lowest: 4.52378617696872
like_count Fixed :4.52378617696872
possibly_sensitive Normal: 23.02172886644268
23.021728866442682
23.02172886644268
23.02172886644268
Lowest: 23.02172886644268
possibly_sensitive Fixed :23.02172886644268
sentiment_overall Normal: -0.342140086974089
sentiment_negative Normal: 1.938387026309749
1.610750330696882
0.4920130204310432
0.6573140946703464
Lowest: 0.4920130204310432
sentiment_negative Fixed :0.4920130204310432
sentiment_neutral Normal: -0.5007258430345204
sentiment_positive Normal: 0.49301498149619377
interior objects Normal: 18.640294253033943
10.9086101205

,Unnamed: 0,like_count,possibly_sensitive,sentiment_overall,language_en,language_es,language_fr,source_Twitter Web App,source_Twitter for Android,source_Twitter for iPhone
0,1.000000,0.0,0.0,0.685581,1.0,0.0,0.0,0.0,0.0,1.0
1,0.995791,0.0,0.0,0.534939,1.0,0.0,0.0,0.0,0.0,1.0
2,0.995464,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0
3,0.993508,0.0,0.0,0.685581,1.0,0.0,0.0,0.0,1.0,0.0
4,0.992321,0.0,0.0,0.883987,1.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
525,0.007138,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0
526,0.005539,0.0,0.0,0.685581,0.0,0.0,1.0,1.0,0.0,0.0
527,0.001808,0.0,0.0,0.777853,1.0,0.0,0.0,0.0,1.0,0.0
528,0.001234,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0


# Modeling 
(same requirements of the modeling project)
* Generate the best possible model for:
    * Regression
    * Classification
    * Clustering
* You choose which features to keep in the model model
* There is no required level of fit metric. Your task is simply to get the best fit metrics possible--not achieve a certain value. All datasets are different and there is no way to compare them

In [16]:
def regression_selection(df, cols_to_drop=[]):
    import pandas as pd
    import numpy as np
    from sklearn import model_selection
    from sklearn import preprocessing
    import sklearn.neural_network as nn
    from sklearn.linear_model import RidgeCV, LassoCV
    import sklearn.ensemble as se
    import sklearn.tree as tree
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
    from sklearn import gaussian_process
    from sklearn import neighbors
    from sklearn import svm
    import sklearn.linear_model as lm
    import pickle

    df = df.select_dtypes(np.number)  # Remove categorical features first
    y = df.like_count                    # Save the label first
    # Remove the label from the feature list
    X = df.drop(columns=cols_to_drop)

    # Scale/normalize the features
    X = pd.DataFrame(preprocessing.MinMaxScaler(
    ).fit_transform(X), columns=X.columns)

    # Split the data
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=12345)

    # Eyeball the data to make sure it looks right:
    X_train

    fit = {}  # Use this to store each of the fit metrics

    # 1. LINEAR MODELS: assumes normal distribution, homoscedasticity, no multi-collinearity, independence, and no auto-correlation (some exceptions apply)

    # 1.1. Ordinary Least Squares Multiple Linear Regression
    model_ols = lm.LinearRegression()
    model_ols.fit(X_train, y_train)
    fit['OrdinaryLS R'] = model_ols.score(X_test, y_test)

    # 1.2. Ridge Regression: more robust to multi-collinearity
    # adjust this alpha parameter for better results (between 0 and 1)
    model_rr = lm.Ridge(alpha=0.5)
    model_rr.fit(X_train, y_train)
    fit['Ridge R'] = model_rr.score(X_test, y_test)

    # 1.3. Lasso Regression: better for sparse values like RetweetCount where most are zeros but a few have many retweets.
    # adjust this alpha parameter for better results (between 0 and 1)
    model_lr = lm.Lasso(alpha=0.1)
    model_lr.fit(X_train, y_train)
    fit['Lasso R'] = model_lr.score(X_test, y_test)

    # 1.4. Least Angle Regression: good when the number of features is greater than the number of samples
    # adjust this alpha parameter for better results (between 0 and 1)
    model_llr = lm.LassoLars(alpha=0.1)
    model_llr.fit(X_train, y_train)
    fit['LARS Lasso R'] = model_llr.score(X_test, y_test)

    # 1.5. Bayesian Regression: probability based; allows regularization parameters, automatically tuned to data
    model_br = lm.BayesianRidge()
    model_br.fit(X_train, y_train)
    fit['Bayesian R'] = model_br.score(X_test, y_test)

    # SUPPORT VECTOR MACHINES
    # 1.9. SVM: this is the default SVM, parameters can be modified to make this more accurate
    model_svm = svm.SVR()
    model_svm.fit(X_train, y_train)
    fit['SupportVM R'] = model_svm.score(X_test, y_test)

    # 1.10. Linear SVM: Faster than SVM but only considers a linear model
    model_lsvm = svm.LinearSVR()
    model_lsvm.fit(X_train, y_train)
    fit['Linear SVM R'] = model_lsvm.score(X_test, y_test)

    # 1.11. NuSVM:
    model_nusvm = svm.NuSVR()
    model_nusvm.fit(X_train, y_train)
    fit['NuSupportVM R'] = model_nusvm.score(X_test, y_test)

    # STOCHASTIC GRADIENT DESCENT REGRESSION
    # 1.12. SGDRegressor:
    model_sgdr = lm.SGDRegressor()
    model_sgdr.fit(X_train, y_train)
    fit['SGradientD R'] = model_sgdr.score(X_test, y_test)

    # KNN: NEAREST NEIGHBORS REGRESSION

    # 1.13. KNeighborsRegressor:
    # model_knnr = neighbors.KNeighborsRegressor(5, 'uniform')
    # model_knnr.fit(X_train, y_train)
    # fit['KNNeighbors R'] = model_knnr.score(X_test, y_test)

    # 1.14. KNeighborsRegressor:
    # model_knnrd = neighbors.KNeighborsRegressor(8, 'distance')
    # model_knnrd.fit(X_train, y_train)
    # fit['KNNeighborsD R'] = model_knnrd.score(X_test, y_test)

    # GAUSSIAN PROCESS REGRESSION

    # 1.15. GaussianProcessRegressor:
    model_gpr = gaussian_process.GaussianProcessRegressor(
        DotProduct() + WhiteKernel())
    model_gpr.fit(X_train, y_train)
    fit['GaussianP R'] = model_gpr.score(X_test, y_test)

    # Sort and print the dictionary by greatest R squared to least
    r2s = sorted_list_by_value = sorted(fit, key=fit.__getitem__, reverse=True)
    for r2 in r2s:
        print(f'{r2}:\t{fit[r2]}')

    # Select the model with the highest R squared
    print(f'Best Regression Model: {r2s[0]} (R2:{fit[r2s[0]]})')
    print('_____________________________________________')
    model = fit[r2s[1]]
    type(model)

    # Save the model with the highest fit metric
    pickle.dump(model, open('regression_model.sav', 'wb'))

def classification_selection(df, cols_to_drop=[]):
    import pandas as pd
    import numpy as np
    from sklearn import model_selection
    from sklearn import preprocessing
    import sklearn.neural_network as nn
    from sklearn.linear_model import RidgeCV, LassoCV
    import sklearn.ensemble as se
    import sklearn.tree as tree
    from sklearn import svm
    import pickle

    df = df.select_dtypes(np.number)  # Remove categorical features first
    y = df.like_count                    # Save the label first
    # Remove the label from the feature list
    X = df.drop(columns=cols_to_drop)

    # Scale/normalize the features
    X = pd.DataFrame(preprocessing.MinMaxScaler(
    ).fit_transform(X), columns=X.columns)

    # Split the data
    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=0.3, random_state=12345)

    # Eyeball the data to make sure it looks right:
    X_train

    fit = {}  # Use this to store each of the fit metrics
    # DECISION TREE MODELS: no assumptions about the data

    # 1.16. Decision Tree Regression
    model_dt = tree.DecisionTreeRegressor(random_state=12345)
    model_dt.fit(X_train, y_train)
    fit['Dec Tree R'] = model_dt.score(X_test, y_test)

    # DECISION TREE-BASED ENSEMBLE MODELS: great for minimizing overfitting, these are based on averaging many unique sub-samples and combining algorithms
    # 1.17. Decision Forrest
    model_df = se.RandomForestRegressor(random_state=12345)
    model_df.fit(X_train, y_train)
    fit['Dec Forest R'] = model_df.score(X_test, y_test)

    # 1.18. ExtraTreesRegressor
    model_etr = se.ExtraTreesRegressor(random_state=12345)
    model_etr.fit(X_train, y_train)
    fit['Extra Trees R'] = model_etr.score(X_test, y_test)

    # 1.19. AdaBoostRegressor
    model_abr = se.AdaBoostRegressor(n_estimators=100, random_state=12345)
    model_abr.fit(X_train, y_train)
    fit['AdaBoost DT R'] = model_abr.score(X_test, y_test)

    # 1.20. GradientBoostingRegressor
    model_gbr = se.GradientBoostingRegressor(random_state=12345)
    model_gbr.fit(X_train, y_train)
    fit['Grad. Boost R'] = model_gbr.score(X_test, y_test)

    # 1.22. VotingRegressor: will combine other algorithms into an average; kind of cool
    model_vr = se.VotingRegressor(estimators=[('DT', model_dt), ('DF', model_df), (
        'ETR', model_etr), ('ABR', model_abr), ('GBR', model_gbr)])
    model_vr.fit(X_train, y_train)
    fit['Voting R'] = model_vr.score(X_test, y_test)

    # 1.23. StackingRegressor
    estimators = [('ridge', RidgeCV()), ('lasso', LassoCV(
        random_state=42)), ('svr', svm.SVR(C=1, gamma=1e-6))]
    model_sr = se.StackingRegressor(
        estimators=estimators, final_estimator=se.GradientBoostingRegressor(random_state=12345))
    model_sr.fit(X_train, y_train)
    fit['Stacking R'] = model_sr.score(X_test, y_test)

    # NEURAL-NETWORK MODELS: Based on deep learning methods

    # 1.24. MLPRegressor
    # Turn max_iter way up or down to get a more accurate result
    model_nn = nn.MLPRegressor(max_iter=1000, random_state=12345)
    model_nn.fit(X_train, y_train)
    fit['NeuralNet R'] = model_nn.score(X_test, y_test)

    # Sort and print the dictionary by greatest R squared to least
    r2s = sorted_list_by_value = sorted(fit, key=fit.__getitem__, reverse=True)
    for r2 in r2s:
        print(f'{r2}:\t{fit[r2]}')

    # Select the model with the highest R squared
    print(f'Best Classification Model: {r2s[0]} (R2:{fit[r2s[0]]})')
    print('_____________________________________________')
    model = fit[r2s[1]]
    type(model)

    # Save the model with the highest fit metric
    pickle.dump(model, open('classification_model.sav', 'wb'))


def clustering(df, cols_to_drop=[]):
    import gower
    from sklearn.cluster import AgglomerativeClustering
    import pickle

    distance_matrix = gower.gower_matrix(df)
    agg = AgglomerativeClustering(
        affinity='precomputed', linkage='average').fit(distance_matrix)

    # make a cluster column
    df_wcluster = df.copy()
    df_wcluster['cluster'] = agg.labels_
    df_wcluster.head()

    # Save the model with the highest fit metric
    pickle.dump(df_wcluster, open('clustering_model.sav', 'wb'))

    return df_wcluster


In [17]:
df_modeling = df.copy()

regression_selection(df_modeling, cols_to_drop=['like_count'])
classification_selection(df_modeling, cols_to_drop=['like_count'])
df_modeling = clustering(df_modeling)

df_modeling


OrdinaryLS R:	0.03287473883438574
Ridge R:	0.030945752839057872
GaussianP R:	0.02728639391288301
Bayesian R:	0.003404835496348624
Lasso R:	-0.008626894410095787
LARS Lasso R:	-0.008626894410095787
SGradientD R:	-0.02103600603083744
Linear SVM R:	-0.09043137702496784
NuSupportVM R:	-0.09555698215052111
SupportVM R:	-0.1474018402889028
Best Regression Model: OrdinaryLS R (R2:0.03287473883438574)
_____________________________________________
NeuralNet R:	-0.03307112986645189
Stacking R:	-0.11818322316519803
Dec Forest R:	-0.18751453797074946
Voting R:	-0.23544792236977163
AdaBoost DT R:	-0.30557563373630336
Extra Trees R:	-0.33665321567811235
Dec Tree R:	-0.6140768890418868
Grad. Boost R:	-0.6425025287579142
Best Classification Model: NeuralNet R (R2:-0.03307112986645189)
_____________________________________________


,Unnamed: 0,like_count,possibly_sensitive,sentiment_overall,language_en,language_es,language_fr,source_Twitter Web App,source_Twitter for Android,source_Twitter for iPhone,cluster
0,1.000000,0.0,0.0,0.685581,1.0,0.0,0.0,0.0,0.0,1.0,0
1,0.995791,0.0,0.0,0.534939,1.0,0.0,0.0,0.0,0.0,1.0,0
2,0.995464,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0,0
3,0.993508,0.0,0.0,0.685581,1.0,0.0,0.0,0.0,1.0,0.0,0
4,0.992321,0.0,0.0,0.883987,1.0,0.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...
525,0.007138,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0,0
526,0.005539,0.0,0.0,0.685581,0.0,0.0,1.0,1.0,0.0,0.0,0
527,0.001808,0.0,0.0,0.777853,1.0,0.0,0.0,0.0,1.0,0.0,0
528,0.001234,0.0,0.0,0.449009,0.0,0.0,0.0,0.0,1.0,0.0,0


# Automation
* Dynamically select the best algorithm for your regression and classification models
* Demonstrate through the feature importance metric which features should be included in the model. However, you don't need to set up an automated selection of features. You can manually decide which features to include
* Dynamically save the best fitting model to a .sav file in the same folder as your .ipynb
* Arrange your .ipynb file so that the "Run all" command will handle all steps above in order from data collection to .sav file